In [1]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer

def init_random_num(qc, reg):
    """Randomly initializes an n-qubit quantum register and prints the register label and the binary number."""
    n = len(reg)  # number of qubits in the register
    binary_representation = ''
    
    for qubit in range(n-1): # choose to leave MSB as 0
        # Randomly apply X gate to flip the qubit with 50% probability
        if np.random.rand() > 0.5:
            qc.x(reg[qubit])  # Apply X gate to the quantum circuit
            binary_representation = '1' + binary_representation
        else:
            binary_representation = '0' + binary_representation
            
    binary_representation = '0' + binary_representation
    decimal_value = int(binary_representation, 2)
    
    # Print the label and the binary number created
    print(f"{reg.name}: {binary_representation} (decimal: {decimal_value})") # choose to leave MSB as 0
    
    return qc


In [5]:
""" This function performs a 2's complement """
def twos(qc, x, c):
    """ This function performs returns x in 2's complement form
    
        Parameters:
        - qc: QuantumCircuit on which the addition is performed
        - x: QuantumRegister for the number to be converted into 2s

        Returns:
        - QuantumCircuit: The updated quantum circuit where x is now in 2's complement
    """
    def increment(qc, x, c):
        """ This function performs a controlled 2's complement with 
            NO overflow handling
        
            Parameters:
            - qc: QuantumCircuit on which the addition is performed
            - x: QuantumRegister for the number to be incremented
            - ctrl: the control qubit for performing the operation
    
            Returns:
            - QuantumCircuit: The updated quantum circuit where x has been incremented 
                conditioned on the ctrl
        """
        n=len(x)
    
        qc.cx(x[0], c[0])
    
        #apply TempAND computes
        for i in range(1,n-1):
            qc.ccx(c[i-1], x[i], c[i])
    
        #end segment
        qc.cx(c[n-2],x[n-1])
    
        # TempAND un-computes and carry increments
        for i in range(n-2,0,-1): # i goes from n-2 to 1
            qc.ccx(c[i-1], x[i], c[i])
            qc.cx(c[i-1], x[i])
    
        qc.cx(x[0], c[0])
        qc.x(x[0])
        
        
        return qc

    n=len(x)

    # flip of x
    for i in range(0,n):
        qc.x(x[i])

    # conditionally increment by 1
    qc = increment(qc, x, c)
    
    return qc

In [8]:
""" MAIN """

# Number of bits
n = 5

ancillaries = QuantumRegister(n, 'ancillaries')
x = QuantumRegister(n, 'x')

# Define a classical register with 2n bits (n for a and n for b)
result = ClassicalRegister(n, 'result')

qc = QuantumCircuit(x, ancillaries, result)

"""initialize the quantum numbers"""
qc = init_random_num(qc, x)


#apply the 2's complement operation
qc=twos(qc, x, ancillaries)

# Measure the sum qubits and carry qubit
#qc.measure_all()

# Measure the qubits into the classical register
qc.measure(x, result[:n])      # Measure 'a' into the first n classical bits


# For execution
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
sim_result = simulator.run(compiled_circuit).result()
counts = sim_result.get_counts()

print(counts)

print("Measurement results:")
for bitstring, count in counts.items():
    # bitstring is of the form 'result' where the first n bits are for 'a' and the last n bits are for 'b'
    x_result = bitstring  # First n bits for a
    #x_decimal_value = int(x_result, 2)

    x_decimal_value = -1 * (2**(n-1))
    for i in range(1, n):
        x_decimal_value += int(x_result[i]) * (2**(n-1-i))

           
    print(f"x = {x_result} (decimal: {x_decimal_value})")


x: 01001 (decimal: 9)
{'10111': 1024}
Measurement results:
x = 10111 (decimal: -9)
